In [ ]:
# default_exp distance

# distance
> find driving distance using bing and osrm api

In [ ]:
#hide
from nbdev.showdoc import *


In [ ]:
#export
from beartype import beartype
from typing import List, Dict, Tuple
import pandas as pd
import yaml, requests

# osrm

In [ ]:
#export
@beartype
def getDistanceOsrm(lat1:float, lon1:float, lat2:float, lon2:float, 
                    osrmEndpoint = 'http://router.project-osrm.org')->float:
  '''
    get driving distance between 2 points using osrm url
  '''
  url = f"{osrmEndpoint}/route/v1/bike/{lon1},{lat1};{lon2},{lat2}?overview=false"
  r = requests.get(url)
  try:
    distance = r.json()['routes'][0]['distance']
    return float(distance)
  except KeyError as e:
    raise KeyError(f"error getting distance got {r.json()} from urll \n{e}")
  

In [ ]:
%%time
#example
testData = '''
lat1 : 13.732048
lon1 : 100.567623
lat2 : 13.856271
lon2 : 100.546467
'''
i = yaml.load(testData, Loader=yaml.FullLoader)
f'the distance is {getDistanceOsrm(**i)} m'

CPU times: user 5.17 ms, sys: 260 µs, total: 5.43 ms
Wall time: 881 ms


'the distance is 17608.5 m'

# get distance bing

In [ ]:
#export
@beartype
def getDistBing(origin:Tuple[float,float]
                ,destinations:List[Tuple[float,float]],
                 bingApiKey = '')->pd.DataFrame:
  '''
    accept a list of tuple origins and destinations, return a dataframe
    input:
      origin :: Tuple[float,float] : (lat,long) of the origin
      destinations:: List[Tuple[float,float]]: [(lat,long),(lat,long)] of the destinations
      bingApiKey:str:: apikey from bing map
    response:
      pd.DataFrame with [dist, dur, lat, long] as indexes, all in float, 
          all distances in m and time in s
  '''
  
  parameters = {
    'origins' : f'{origin[0]},{origin[1]}',
    'destinations' : ';'.join(f'{d[0]},{d[1]}'for d in destinations),
    'travelMode' : 'driving',
    'key': bingApiKey
  }
  url = 'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix'
  r =  requests.get(url,params=parameters).json()
  results:pd.DataFrame = pd.DataFrame(r['resourceSets'][0]['resources'][0]['results'])
  destinationsDf:pd.DataFrame = pd.DataFrame(destinations)
  destinationsDf.columns = ['lat', 'long']
  results = pd.concat([results,destinationsDf],axis=1).reindex(
    ['travelDistance','travelDuration', 'lat','long'], axis = 1)
  results.columns  
#   return results
#   return [(result['travelDistance'], result['travelDuration'])for result in results]
#   return [result['travelDistance'] for result in results], [result['travelDuration'] for result in results]



In [ ]:
# example
from nicHelper.secrets import getSecret
testData = '''
origin:
  !!python/tuple
  - 13.732048
  - 100.567623
destinations:
  -  !!python/tuple
    - 13.856271
    - 100.546467
  -  !!python/tuple
    - 13.856444
    - 100.549487
  - !!python/tuple
    - 13.857444
    - 100.549487
'''
i = yaml.load(testData, Loader = yaml.FullLoader)

apikey = getSecret( name = 'webApiKeys')['bing']
results = getDistBing( **i, bingApiKey = apikey )
results
# print(f'distances are {dist}, time taken are {timetaken}')

,travelDistance,travelDuration,lat,long
0,18.208,25.7167,13.856271,100.546467
1,18.812,27.2167,13.856444,100.549487
2,19.490,29.3833,13.857444,100.549487
